## Positional Encoding (sinusoid)

In [ ]:
# Positional Encoding.
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        self.dropout = nn.Dropout(dropout_p)

        # 주어진 공식대로 적용해주면 된다.
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3,..
        # 10000^(2i/dim_model) 짝수로 .. positions list를 나눠줄건데 sin cos 번갈아가면서 가기 때문인 것 같다.
        division_term = torch.exp(torch.arange(0, dim_model,2).float() * (-math.log(10000.0))/dim_model)
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)

        # Register buffer 로 저장하면 훈련없이 한 레이어로 적용한다.
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1) # (1, max_len, dim_model) -> (max_len, 1, dim_model)
        # 위의 transpose는 batch_first = False 인 경우 elementwise operation 하나부다.

    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])

# 이런식으로 작동하는구나~

# BERT 구현

## WordPiece Tokenizer, BPE (Byte Pair Encoding)

In [1]:
# 하기 전에 우선 WordPiece Tokenizer 부터 공부하자.

# Word Piece -> 단어를 표현할 수 있는 subwords units로 모든 단어를 표현 /
# RNN과 같은 word embedding vectors를 사용 -> 단어 개수만큼 embedding vector를 학습. ㅜㅜ 너무 많아
# -> 제한된 개수의 단어를 사용하자. but long tail 이론에 의거하여 조그마한 단어를 무시함녀 미등록단어 문제 발생.

# 언어는 글자를 subword units으로 사용. (알파벳..) 하지만 이러한 유닛은 개념을 지칭하기는 어렵다.
# 영어에 있어서는 이러한 unit은 모호성이 너무 강하다. --> 잘 나눠서 unit으로 사용하자.

# WPM
# Jet makers feud over seat width with big orders at stake.
# Wordpieces : _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake
# _ -> 문장 복원을 위함.. 시작 단어를 _로 분할한다. Jet이나 feud 와 같은 단어는 자주 등장하지 않아서 띄어쓰기로 또 분할된다.

# recover function
def recover(tokens):
    sent = "".join(tokens)
    sent = sent.replace('_', " ")
    return sent
tokens = ["_J", "et", "_makers", "_fe", "ud", "_over", "_seat", "_width", "_with", "_big", "_orders", "_at", "_stake"]
recover(tokens)

' Jet makers feud over seat width with big orders at stake'

In [2]:
# BPE. Byte-pair Encoding
# for 문으로 가장 빈도수가 많은 bigram을 찾는다. character가 units 이다.

# 배경. 어떤 document에서 단어의 빈도를 구한다. -> 각 단어의 character 사이의 공백 넣고 마지막엔 </w>

# 이 데이터에 대해 stats를 받고 merge 한다.



import re, collections

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq # 우선 bigram 단위로 frequency 확인..
            # merge 후 계속 적용한다.
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair)) # 특수문자를 \로 바꿔준다. (escape 한다라고 함.)
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

num_merges = 10 # 10 번 merge 진행할거임
for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get) # 현재 vocab 가장 좋은 bigram
    vocab = merge_vocab(best, vocab) # vocab merge
    print(best)

print(vocab)
# 이런식으로 WPM 에서의 subwords 개념을 만들어낼 수 있다.



('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


In [3]:
# 구글에서 발표한 sentencepiece 를 사용할 수 있다.
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.3 MB/s 


## sentencepiece 라이브러리 활용

In [4]:
import os
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sentencepiece as spm
path = '/content/drive/MyDrive/Dataset/'
# tokenizer 사용.
parameter = "--input={} --model_prefix={} --vocab_size={} --user_defined_symbols={} --model_type={} --character_coverage={}"

# cmd 형태로 주는데?

train_input_file = path + "BookCorpus_train.txt"
vocab_size = 30000
prefix = 'bookcorpus_spm'
user_defined_symbols = "[PAD],[CLS],[SEP],[MASK]"
model_type = "bpe" # Byte Pair Encoding (Bigram..)
character_coverage = 1.0 # default

cmd = parameter.format(train_input_file, prefix, vocab_size, user_defined_symbols, model_type, character_coverage)

spm.SentencePieceTrainer.Train(cmd) # 주어진 corpus로 tokenizer를 훈련시킨다. heuristic 한 방법이다.

## Dataset!

In [6]:
import random
# Dataset 만드는건 torchtext dataset을 사용할 수 있지만 곧 사라진다.
# torch.utils.data.Dataset 사용해서 커스텀 데이터셋 만들자.


"""
    BPE(Byte Pair Encoding) 방법으로 pretrained vocab을 가져와서 list[str] 형 데이터에 적용시켜줄거임.

    특정 토큰으로
    MLM 을 위한 [MASK]
    NSP 에서 다음 문장에 대한 [SEP]
    sequence length를 맞춰주기위한 패딩 [PAD]
    Classification Token [CLS]
"""

class LanguageModelingDataset(torch.utils.data.Dataset):
    def __init__(self, data, vocab: spm.SentencePieceProcessor, sep_id: str="[SEP]", cls_id: str='[CLS]',
                 mask_id: str='[MASK]', pad_id: str='[PAD]', seq_len: int=512, mask_frac: float=0.15, p: float=0.5):
        
        # vocab으로 BPE 적용 
        super(LanguageModelingDataset, self).__init__()
        self.vocab = vocab # SentencePieceProcessor
        self.data = data # language
        self.seq_len = seq_len
        self.sep_id = vocab.piece_to_id(sep_id) # 각 특수 토큰에 대응하는 id 가져오기
        self.cls_id = vocab.piece_to_id(cls_id)
        self.mask_id = vocab.piece_to_id(mask_id)
        self.pad_id = vocab.piece_to_id(pad_id)
        self.p = p
        self.mask_frac = mask_frac

    def __getitem__(self, i):
        seq1 = self.vocab.EncodeAsIds(self.data[i].strip()) # 우선 data 하나 가져온다. sequence 단위
        seq2_idx = i+1
        if random.random() < self.p : # 다음 sequence가 아닌가?.. 확률로 배정
            is_next = torch.tensor(0)
            while seq2_idx != i+1:
                seq2_idx = random.randint(0, len(self.data))
        else :
            is_next = torch.tensor(0)

        seq2 = self.vocab.EncodeAsIds(self.data[seq2_idx])

        if len(seq1) + len(seq2) >= self.seq_len - 3:
            idx = self.seq_len - 3 - len(seq1)
            seq2 = seq2[:idx]  # 크기 제한

        # mask가 적용이 안된 상태의 seq.. -> 사전에 정해둔 sequence length를 맞춰주기 위해서 패딩을 지정해준다.
        mlm_target = torch.tensor([self.cls_id] + seq1 + [self.sep_id] + seq2 + [self.sep_id] + [self.pad_id] * (self.seq_len -3 - len(seq1) - len(seq2))).long().contiguous()
        sent_emb = torch.ones((mlm_target.size(0)))
        _idx = len(seq1) + 2

        # sentence embedding : 0 -> A, 1 -> B
        sent_emb[:_idx] = 0

        def masking(data):
            data = torch.tensor(data).long().contiguous()
            data_len = data.size(0)
            ones_num = int(data_len * self.mask_frac) # masking 갯수
            zeros_num = data_len - ones_num # non masked
            lm_mask = torch.cat([torch.zeros(zeros_num), torch.ones(ones_num)])
            lm_mask = lm_mask[torch.randperm(data_len)]  # data length 만큼 arange 이후 셔플.. 결국 lm mask 셔플한거임.
            data = data.masked_fill(lm_mask.bool(), self.mask_id) # data에 lm_mask가 True인 경우 (mask 적용하는 경우) 
            # mask_id 로 채워버린다.

            return data
        # token은 놔두고 각 sequence에 대해 마스킹 적용 
        mlm_train = torch.cat([torch.tensor([self.cls_id]), masking(seq1), torch.tensor([self.sep_id]), masking(seq1), torch.tensor([self.sep_id])]).long().contiguous()
        # 크기 맞춰주기 위해 패딩 넣어주기.
        mlm_train = torch.cat([mlm_train, torch.tensor([self.pad_id] * (512 - mlm_train.size(0)))]).long().contiguous()

        return mlm_train, mlm_target, sent_emb, is_next

    def __len__(self):
        return len(self.data) -1 # NSP에서 마지막 문장은 안된다. -> 마지막 문장에서 -1 idx에 대해서만 사용 가능. DataLoader에서 getitem 접근할 때 idx 부여를 len(data)를 기반으로 적용하기 때문에 이렇게 놔두자.
    
    def __iter__(self):
        for x in self.data:
            yield x
    
    def get_vocab(self):
        return self.vocab
    
    def decode(self, x):
        return self.vocab.DecodeIds(x)

## 실행되는지만 확인하려고 BookCorpus data를 쪼개서 가져왔음

In [7]:
file = "/content/drive/MyDrive/Dataset/BookCorpus_train.txt"
data = []
data_len = 8000 # Colab RAM 제한 문제. 일부만 가져오자!
with open(file) as f: 
    for i, line in enumerate(f.readlines()):
        if i >= data_len:
            break
        data.append(line.strip()) # txt 파일 line 읽기
        
    

In [8]:
vocab_file = "bookcorpus_spm.model" # pretrained vocab (BPE) 위에서 spm train 시키면 spm.model 파일 생긴다.
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)
dataset = LanguageModelingDataset(data=data, vocab=vocab)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)

In [9]:
for batch, (mlm_train, mlm_target, sent_emb, is_next) in enumerate(dataloader):
    print(mlm_train.size())
    print(mlm_target.size())
    print(sent_emb.size())
    print(is_next.size())
    break

torch.Size([32, 512])
torch.Size([32, 512])
torch.Size([32, 512])
torch.Size([32])


## Bert Model

In [ ]:
"""
    주의할점 _ Batch First 상태이다. 데이터를 그렇게 불러오는중.
"""

def get_attn_pad_mask(seq_q, seq_k, pad_index):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(pad_index).unsqueeze(1)
    pad_attn_mask = torch.as_tensor(pad_attn_mask, dtype=torch.float)
    return pad_attn_mask.expand(batch_size, len_q, len_k) # batch first

class PositionalEncoding(nn.Module):
    def __init__(self, d_model=768, seq_len=512, dropout=0.1):
        super().__init__()
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        self.emb = nn.Embedding(seq_len, d_model) # 위치에 따른 고유값.

    def forward(self, x):
        pos = torch.arange(self.seq_len, dtype=torch.long, device=x.device)
        pos = pos.unsqueeze(0).expand(x.size())
        pos_emb = self.emb(pos)
        return self.dropout(pos_emb)


class BERTEmbedding(nn.Module):
    def __init__(self, seq_len=512, voc_size=30000, d_model=768, dropout=0.1):
        super().__init__()
        self.tok_emb = nn.Embedding(num_embeddings=voc_size, embedding_dim=d_model) # 30000개 단어 포함하였기 때문 
        # 30000개의 고유값을 768에 욱여넣기. B, S, d_model shape
        self.tok_dropout = nn.Dropout(dropout)
        self.seg_emb = nn.Embedding(num_embeddings=2, embedding_dim=d_model)
        self.seg_dropout = nn.Dropout(dropout)
        self.pos_emb = PositionalEncoding(d_model, seq_len, dropout)


    def forward(self, data, seg_emb):
        tok_emb = self.tok_emb(data) #  B, S, d_model shape
        seg_emb = self.seg_emb(seg_emb) #  B, S, d_model shape
        pos_emb = self.pos_emb(data) #  B, S, d_model shape 

        return self.tok_dropout(tok_emb) + self.seg_dropout(seg_emb) + pos_emb


class BERTModel(nn.Module):
    def __init__(self, voc_size=30000, seq_len=512, d_model=768, d_ff=3072, pad_idx=1, num_encoder=12,
                 num_heads=12, dropout=0.1):
        super().__init__()
        self.pad_idx = pad_idx
        self.emb = BERTEmbedding(seq_len, voc_size, d_model, dropout)
        encoder = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=d_ff, dropout=dropout, batch_first=True)
        self.encoders = nn.TransformerEncoder(encoder, num_layers=num_encoder)

        self.mlm = nn.Linear(d_model, voc_size) # MLM Task classification 처럼 ..
        self.nsp = nn.Linear(d_model, 2) # NSP Task classificiation 처럼..
        

    def forward(self, input, seg):
        pad_mask = get_attn_pad_mask(input, input, self.pad_idx) 

        emb = self.emb(input, seg) # ( B, S, d_model ) shape

        # mask는 batch size 없이 S, S 형태로만 들어가야한다.
        feat = self.encoders(emb, pad_mask[0])# ( B, S, d_model ) shape
    
        
        mlm = self.mlm(feat) # B, S, D_model
        nsp = self.nsp(feat)
        return mlm, nsp[:,0,:] # NSP : (B, 2), Class token만 사용하기 때문.





### 실행은 대략 이런식으로 ..

모델이 너무 커서 코랩 무료 버전으로 돌아가질 않는다.  
그래서 대략만 짜고 넘어간다.  
data가 어떤 shape을 가지고, 어떤 형태로 흘러들어가는지, 모델을 거치면서 어떤 operation이 적용되는 정도를 파악하기 위해 작성했다.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BERTModel().to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

criterion = nn.CrossEntropyLoss()

for epoch in range(1, 101):
    m_losses = []
    n_losses = []
    print(f'epoch : {epoch}/100')
    for batch, (mlm_train, mlm_target, sent_emb, is_next) in enumerate(dataloader):
        sent_emb = sent_emb.long()
        mlm_train, mlm_target, sent_emb, is_next = mlm_train.to(device), mlm_target.to(device), sent_emb.to(device), is_next.to(device)

        mlm, nsp = model(mlm_train, sent_emb) # 허허..

        m_loss = criterion(mlm, mlm_target)
        n_loss = criterion(nsp, is_next)

        m_losses.append(m_loss.item())
        n_losses.append(n_loss.item())

        loss = m_loss + n_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"MLM : {np.mean(m_losses)}, NSP : {np.mean(n_losses)}") # 대충 이런식으로 pretrain이 진행된다!

